# Preprocessing, Pipeline and Model

### Partyequipment

In [1]:
#normal stuff
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler
sns.set()
%matplotlib inline

#special effects
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lars, ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, RobustScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error

### Partyguests

In [2]:
#vspath
path = r"data\train.csv"
X_raw = pd.read_csv(path, index_col="Id")
X_ana = X_raw.copy()
X_test = pd.read_csv(path, index_col ="Id")
print(X_raw.columns)

#kaggle path
#X_raw = pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv', index_col='Id')
#X_test = pd.read_csv('/kaggle/input/home-data-for-ml-course/test.csv', index_col='Id')

X = X_raw.copy()

neighborhood_ordered = list(X.groupby("Neighborhood")["SalePrice"].mean().sort_values().index)

X.dropna(subset=["SalePrice"], axis=0, inplace=True)
#X.dropna(axis=1, inplace = True)
y = np.log(X.SalePrice)
print(y)
X.drop("SalePrice", axis=1, inplace=True)

print(X.head())
print(X.info())
print(X.describe())

#X["MSSubClass"] = X["MSSubClass"].astype("category")
#X_test["MSSubClass"] = X_test["MSSubClass"].astype("category")

col_obj = X.select_dtypes(include=["object", "bool","category"]).columns
col_obj_above_10 = list([col for col in col_obj if X[col].nunique() > 15])
col_obj_below_10 = list(set(col_obj)-set(col_obj_above_10))
col_num = list(X.select_dtypes(exclude=["object", "bool","category"]).columns)
col_with_nan = [col for col in X.columns if X[col].isna().any()]


print("Summary:\n")
print("Number of columns: {}".format(len(X.columns)))
print("Number of object columns: {}".format(len(col_obj)))
print("Number of columns with NaNs: {}".format(len(col_with_nan)))
print("Number of object columns with NaNs: {}".format(len(set(col_with_nan)-set(col_num))))
print("Number of numeric columns with NaNs: {}".format(len(set(col_with_nan)-set(col_obj))))


Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

### Preprocessing


In [3]:

#Ordinal1:  "ExterQual", "ExterCond", "BsmtQual", "BsmtCond", "HeatingQC"
#Ordinal2:  ["BsmtFinType1","BsmtFinType2"]
#Ordinal3: "Electrical"
#Ordinal4: 

#numerical1:  LotFrontage

for c in ["ExterQual", "ExterCond", "BsmtQual", "BsmtCond", "HeatingQC","Electrical","BsmtFinType1","BsmtFinType2"]:
    col_obj_below_10.remove(c)
    
#col_obj_below_10.remove(["ExterQual", "ExterCond", "BsmtQual", "BsmtCond", "HeatingQC","Electrical","BsmtFinType1","BsmtFinType2"])


X["LotFrontage"]= X.groupby(["LotShape", "LotConfig","BldgType"])["LotFrontage"].transform(lambda x: x.fillna(x.median()))
X_test["LotFrontage"] = X.groupby(["LotShape", "LotConfig","BldgType"])["LotFrontage"].transform(lambda x: x.fillna(x.median()))

log_cols = ["GrLivArea"]
for c in log_cols:
    X[c] = np.log(X[c].fillna(X[c].median()))
    X_test[c]= np.log(X_test[c].fillna(X_test[c].median()))


col_obj_below_10.append("MSSubClass")
col_num.remove("LotFrontage")

for c in  X.select_dtypes(include=["object"]).columns:
    X[c].fillna("None")
    X_test[c].fillna("None")

res= {}
np.random.seed(1)

### Pipeline and Model

In [4]:

#reg = ElasticNet()

X_train, X_valid, y_train, y_valid = train_test_split( X, y, train_size = 0.7, test_size=0.3, random_state=0)
reg = XGBRegressor( evals=y_valid, eval_metric="mae",cv=False, seed=0, verbosity=1 ) #evals=y_valid, eval_metric="mae",


#numeric_transformer = Pipeline(["imputer",SimpleImputer(strategy="median")])

#log y
#y_train, y_valid = np.log(y_train), np.log(y_valid)





#raising value error due to lack in synchronization caused by OneHotEncoder and OrdinalEncoder
preprocessing = ColumnTransformer(transformers=[("numeric", SimpleImputer(strategy = "median"), col_num) ,
                                                ("LotFrontage_", SimpleImputer(strategy = "median"), ["LotFrontage"]) ,

                                            ("obj_below_10", Pipeline([("obj_imputer1", SimpleImputer(strategy = "most_frequent")),
                                                                        ("OHE"            , OneHotEncoder(handle_unknown='ignore'
                                                                                                            ))
                                                                        ]), col_obj_below_10) ,
                                            ("Neighborhood_", Pipeline([("obj_imputer2", SimpleImputer(strategy = "most_frequent")),
                                                                        ("label_encoder1"  , OrdinalEncoder(categories=[neighborhood_ordered])
                                                                                                            )
                                                                        ]), ["Neighborhood"]),
                                            ("Ratings1_", Pipeline([("obj_imputer3", SimpleImputer(strategy ="constant", fill_value="TA")),
                                                                        ("label_encoder2"  , OrdinalEncoder(categories=[["Po", "Fa", "TA", "Gd", "Ex"],
                                                                                                                        ["Po", "Fa", "TA", "Gd", "Ex"],
                                                                                                                        ["Po", "Fa", "TA", "Gd", "Ex"],
                                                                                                                        ["Po", "Fa", "TA", "Gd", "Ex"],
                                                                                                                        ["Po", "Fa", "TA", "Gd", "Ex"]])
                                                                                                            )
                                                                        ]), [ "ExterQual", "ExterCond", "BsmtQual", "BsmtCond", "HeatingQC"]),
                                            ("Ratings2_", Pipeline([("obj_imputer4", SimpleImputer(strategy = "most_frequent")),
                                                                        ("label_encoder3"  , OrdinalEncoder(categories=[["GLQ","ALQ","BLQ","Rec","LwQ","Unf","NA"],
                                                                                                                        ["GLQ","ALQ","BLQ","Rec","LwQ","Unf","NA"]])
                                                                                                            )
                                                                        ]), ["BsmtFinType1","BsmtFinType2"]),
                                            ("Electric_", Pipeline([("obj_imputer5", SimpleImputer(strategy = "most_frequent")),
                                                                        ("label_encoder4"  , OrdinalEncoder(categories=[["Mix", "FuseP", "FuseF", "FuseA", "SBrkr"]])
                                                                                                            )
                                                                        ]), ["Electrical"]),
                                            ("Exterior_2nd", Pipeline([("obj_imputer7", SimpleImputer(strategy = "most_frequent")),
                                                                        ("label_encoderX"  , OrdinalEncoder(categories=[list(X["Exterior2nd"].unique())]))
                                                                        ]), ["Exterior2nd"])], remainder='drop')


my_pipe = Pipeline([("preprocessing",preprocessing),
                     #("pca", PCA(n_components = 35)),
                     ("rb_scaler", RobustScaler()),
                     ("reg",reg)])

param_grid = {  #"preprocessing__numeric__strategy":["median", "mean"],
                #"pca__n_components": [31],
                "reg__early_stopping_counts": [10],
                "reg__n_estimators": [511],#done
                "reg__learning_rate":[0.02428125],
                "reg__max_depth": [5],#done
                "reg__min_child_weight":[4],#done[1, 2, 3, 4, 5, 6, 7, 8], 
                "reg__gamma":[0], #done[ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
                "reg__subsample":[0.58],#done[0.940136255964662][0.3283798423145866]
                "reg__colsample_bytree": [0.5], #done[i/100.0 for i in range(45,55,1)], 
                "reg__alpha":[0.10925528556796542], #done
                
                }


cv = 3




search = GridSearchCV(my_pipe, param_grid=param_grid, scoring="neg_mean_absolute_error", verbose=50, n_jobs=-2, cv = cv,)
search.fit(X_train,y_train)
print(search.best_score_)
#print("best iteration: ", search.best_iteration, s)
search_pred = search.predict(X_valid)
#search_pred = np.exp(search_pred)
search_score = mean_absolute_error(np.exp(y_valid), np.exp(search_pred))
print(search.best_params_)
#res[search.best_score_] = search.best_params_

print(search_score)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   1 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:    4.3s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:    4.3s finished
-0.08444517040227473
{'reg__alpha': 0.10925528556796542, 'reg__colsample_bytree': 0.5, 'reg__early_stopping_counts': 10, 'reg__gamma': 0, 'reg__learning_rate': 0.02428125, 'reg__max_depth': 5, 'reg__min_child_weight': 4, 'reg__n_estimators': 511, 'reg__subsample': 0.58}
15073.62522295947


In [5]:
model = search.best_estimator_
model

Pipeline(memory=None,
         steps=[('preprocessing',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('numeric',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='median',
                                                                verbose=0),
                                                  ['MSSubClass', 'LotArea',
                                                   'OverallQual', 'OverallCond',
                                                   'YearBu

In [6]:
param_grid = {  #"preprocessing__numeric__strategy":["median", "mean"],
                #"pca__n_components": [31],
                "reg__early_stopping_counts": [50],
                "reg__n_estimators": [171],#done
                "reg__learning_rate":[0.1],
                "reg__max_depth": [4], #done[ 3, 4, 5, 6, 8, 10, 12, 15],
                "reg__min_child_weight":[3], #done[1, 2, 3, 4, 5, 6, 7, 8], 
                "reg__gamma":[1], #done[ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
                "reg__subsample":[0.940136255964662],#done[0.940136255964662]
                "reg__colsample_bytree": [0.5], #done[i/100.0 for i in range(45,55,1)], 
                "reg__alpha":[0.10925528556796542], #done
                
                }

In [7]:
XGBR_final = XGBRegressor(n_estimators=4500, learning_rate=0.008, max_depth = 4, min_child_weight=4, gamma=0, subsample= 0.940136255964662, colsample_bytree = 0.5, alpha = 0.10925528556796542, early_stopping_counts=50, evals=y_valid, eval_metric="mae",cv=True, seed=0, verbosity=1 )


my_pipe_out = Pipeline([("preprocessing", preprocessing),
                        ("rb_scaler", RobustScaler()),
                        ("reg_final", XGBR_final)
                        ])


my_pipe_out.fit(X_train, y_train)

valid_preds = my_pipe_out.predict(X_valid)
valid_score = mean_absolute_error(np.exp(y_valid), np.exp(valid_preds))
print(valid_score)


14847.06650435217


225


### Output generation


In [10]:
test_preds = my_pipe_out.predict(X_test)

#recalc preds
test_preds=np.exp(test_preds)
output = pd.DataFrame({"Id": X_test.index,
                     "SalePrice": test_preds})
output.to_csv("submission_other.csv", index=False)

param_grid = {  #"preprocessing__numeric__strategy":["median", "mean"],
                #"pca__n_components": np.linspace(25, 65, 5, dtype="int"),
                "reg__early_stopping_counts": [50],
                "reg__n_estimators": [342],
                "reg__learning_rate":[0.04002371183530562],
                "reg__max_depth": [5],
                "reg__min_child_weight": [1.02],
                "reg__gamma":[0],
                "reg__subsample":[0.8264488486588846],
                "reg__colsample_bytree": [0.551988011711525],
                "reg__alpha":[0.107],
                
                }